In [11]:
import numpy as np
from sklearn.datasets import make_blobs
from sklearn.model_selection import train_test_split

## Generating Dataset  

In [12]:
data = make_blobs(n_samples=10000, n_features=4,centers=2,random_state=5)

features = data[0]
labels   = data[1]

#### Splitting the Dataset

Here we use function from sklearn Library to split our data into train dataset to train our model on it and test dataset to test our model after training

In [13]:
features_train,features_test,labels_train,labels_test = train_test_split(features,labels,test_size=0.30, random_state=1991)
labels_train = labels_train.reshape(1,len(labels_train))
labels_test = labels_test.reshape(1,len(labels_test))

#### Print the shape of train as well as test dataset 

In [14]:
print("Your training dataset consist of {} samples and {} features and the shape = ".format(features_train.shape[0],features_train.shape[1]), features_train.shape)
print("\nYour test dataset consist of {} samples and {} features and the shape = ".format(features_test.shape[0],features_test.shape[1]), features_test.shape)



Your training dataset consist of 7000 samples and 4 features and the shape =  (7000, 4)

Your test dataset consist of 3000 samples and 4 features and the shape =  (3000, 4)


# Try to build a neural network with 2 hidden layers, each containing (8) neurons and output Layer with single neuron
Activation Function : Sigmoid

Loss Function : Least Square Error 

## Exersise 1.A

##### Initialize the weight and bias for each Hidden Layer as well as output Layer 

In [15]:
np.random.seed(1991)
# I will initialize the first Hidden Layer Weight & bias for You

weight_h1 = np.random.rand(32,4)  ## 4 : number of features (number of inputs in the first Hidden Layer)   &&  32 : number of neurons in the first Hidden layer
bias_h1   = np.random.rand(32,1)  ## 32 : number of neurons in the first Hidden Layer 


##### Write Your Code Here ####
weight_h2 = np.random.rand(64,32)
bias_h2   = np.random.rand(64,1)


weight_o3 = np.random.rand(1,64)
bias_o3   = np.random.rand(1,1)

# then save all weights in one variable and all bias in one variable
weight = (weight_h1,weight_h2,weight_o3)
bias   = (bias_h1,bias_h2,bias_o3)

# you can get weight_h1 for example like this   
# weight_h1 = weight[0]   

## Exersise 1.B

Write a Class that contains 4 Functions ( sigmoid , loss function , forward_propagation, backward_propagation)


Least square Error = 
<img src="https://latex.codecogs.com/gif.latex?(1/n)&space;*&space;\sum&space;(labels&space;-&space;pred)^{2}" title="(1/n) * \sum (labels - pred)^{2}" />

n = number of samples ... in your case it's 700

In [16]:
class Deep_Neural_Network():
    
    def __init__(self,features_train,labels_train,weight,bias,learning_rate):
        
        self.features_train = features_train
        self.labels_train   = labels_train
        self.weight         = weight
        self.bias           = bias
        self.learning_rate  = learning_rate
        
    def sigmoid(self,z):
        
        return 1/(1+np.exp(-z))
    
    def cross_enthropy(self,labels_train,y_pred):
        
        loss = -labels_train*np.log(y_pred) - (1-labels_train)*np.log(1-y_pred)
        cost = np.mean(loss)
        
        return cost
    def least_square_error(self,labels_train,y_pred):
        
        return np.mean(np.square(labels_train-y_pred))
        
        
    def forward_propagation(self):
        ### Here you will implement the feedforward of the Neural_Network
        first_hidden_layer_out  = np.add(np.dot(self.weight[0],self.features_train.T) , self.bias[0])
        first_activation        = self.sigmoid(first_hidden_layer_out)
        ### Write your code here  ###
        
        second_hidden_layer_out = np.add(np.dot(self.weight[1],first_activation) , self.bias[1])
        second_activation       = self.sigmoid(second_hidden_layer_out)
        
        output_layer            = np.add(np.dot(self.weight[2],second_activation) , self.bias[2])
        output_activation       = self.sigmoid(output_layer)
        
        y_pred = output_activation        
        ### compute the loss
        
        loss = self.least_square_error(self.labels_train,y_pred)
        
        cache = ((first_hidden_layer_out,second_hidden_layer_out,output_layer),(first_activation,second_activation,y_pred))
        return y_pred,loss,cache
        
        
        
        
    def backward_propagation(self,y_pred,cache):
        
        dy_pred                  = -2*(self.labels_train - y_pred)
        doutput_layer            = dy_pred * (y_pred*(1-y_pred))
        dweight_o3               = np.dot(doutput_layer,cache[0][1].T)/self.features_train.shape[0]
        dbias_o3                 = np.mean(doutput_layer,axis=1)
        
        dsecond_activation       = np.dot(self.weight[2].T,cache[0][2])
        dsecond_hidden_layer_out = dsecond_activation * (cache[1][1]* (1- cache[1][1])) 
        dweight_h2               = np.dot(dsecond_hidden_layer_out,cache[1][0].T)/self.features_train.shape[0]
        dbias_h2                 = np.mean(dsecond_hidden_layer_out,axis=1)        
        
        dfirst_activation        = np.dot(self.weight[1].T,cache[0][1])
        dfirst_hidden_layer_out = dfirst_activation * (cache[1][0]* (1- cache[1][0])) 
        dweight_h1               = np.dot(dfirst_hidden_layer_out,self.features_train)/self.features_train.shape[0]
        dbias_h1                 = np.mean(dfirst_hidden_layer_out,axis=1) 
        ##### update your weight and bias (weight_h1,weight_h2,weight_o3......bias_h1,bias_h2,bias_o3)
        
        weight_h1,weight_h2,weight_o3 = self.weight
        bias_h1,bias_h2,bias_o3       = self.bias
        weight_h1 -= self.learning_rate * dweight_h1
        weight_h2 -= self.learning_rate * dweight_h2
        weight_o3 -= self.learning_rate * dweight_o3
        
        bias_h1   -= self.learning_rate * dbias_h1.reshape(32,1)
        bias_h2   -= self.learning_rate * dbias_h2.reshape(64,1)
        bias_o3   -= self.learning_rate * dbias_o3
        
        self.weight = (weight_h1,weight_h2,weight_o3)
        self.bias   = (bias_h1,bias_h2,bias_o3)
        
        
        
        
    

In [17]:
model = Deep_Neural_Network(features_train,labels_train,weight,bias,learning_rate=0.001)


In [18]:
# run this cell to train your model :D
epochs = 15000
for epochs in range(epochs):
        y_pred, loss, cache = model.forward_propagation()
        model.backward_propagation(y_pred,cache)
        if (epochs % 5000 == 0):
            print(loss)

0.5004285714285195
0.46631369553840607
0.14178700740842895


# Here You can test your model and see tha accuracy :D

you just need to run both cells to see how good is your model and Good Luck 

In [19]:
def test(features_test,labels_test):
    
    h1  = np.add(np.dot(model.weight[0],features_test.T) , model.bias[0])
    a1  = model.sigmoid(h1)
        
    h2  = np.add(np.dot(model.weight[1],a1) , model.bias[1])
    a2  = model.sigmoid(h2)
        
    out = np.add(np.dot(model.weight[2],a2) , model.bias[2])
    a3  = model.sigmoid(out)
        
    y_pred = a3
                
    loss = model.least_square_error(labels_test,y_pred)
    
    correct_answer = 0
    
    for i in range(features_test.shape[0]):
        if (labels_test[0][i] == np.round(y_pred[0][i])):
            correct_answer +=1
            
    
    accuracy = (correct_answer / features_test.shape[0]) * 100
    
    return accuracy
        
    

In [20]:
accuracy = test(features_test,labels_test)
print("The accuracy of your model is {}".format(accuracy))

The accuracy of your model is 100.0
